<a href="https://colab.research.google.com/github/adampotton/MDM3-Rep-3/blob/main/both_imagery_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import files, drive
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### S2 model process

In [2]:
table = pd.read_csv('/content/drive/My Drive/S2 Data/labels.csv')
mapping = pd.read_csv('/content/drive/My Drive/S2 Data/tree_mapping.csv')
df = pd.DataFrame(mapping)
numbers = np.load('/content/drive/My Drive/S2 Data All/numbers.npy')
arrays = np.load('/content/drive/My Drive/S2 Data All/arrays.npy')
labels = np.load('/content/drive/My Drive/S2 Data All/labels.npy')

def extract_numbers(filename):
    return filename.split('_')[-3].replace('.tif', '')

table['File Number'] = table['File Name'].apply(extract_numbers)

tol = 0.99

table_numeric = table.apply(pd.to_numeric, errors='coerce')
columns_to_check = table_numeric.columns.difference(['File Number'])
filtered_table = table_numeric[(table_numeric[columns_to_check] >= tol).any(axis=1)]

indexes = np.where(np.isin(numbers, filtered_table['File Number']))[0]

arrays_S2 = arrays[indexes]
labels_S2 = labels[indexes]
numbers_S2 = numbers[indexes]
numbers_S2 = np.array(numbers_S2, dtype=int)


### Aerial imagery

In [3]:
# arrays_A = np.load('/content/drive/My Drive/Aerial Data/aerial_99_images.npy')
arrays_A_fill = np.arange(17618)
labels_A = np.load('/content/drive/My Drive/Aerial Data/aerial_99_labels.npy')
numbers_A = np.load('/content/drive/My Drive/Aerial Data/aerial_99_numbers.npy')

### Getting Data Points

In [ ]:
import random

n = 17000
datapoints = []

for _ in range(n):
    number_index = random.randint(0, len(numbers_A) - 1)

    number_A = numbers_A[number_index]
    label_A = labels_A[number_index]
    array_A = arrays_A_fill[number_index]

    index_S2 = np.where(numbers_S2 == number_A)[0]

    if len(index_S2) == 0:
        continue

    index_S2 = index_S2[0]
    label_S2 = labels_S2[index_S2]
    array_S2 = arrays_S2[index_S2]

    numbers_S2 = np.delete(numbers_S2, index_S2)
    labels_S2 = np.delete(labels_S2, index_S2)
    arrays_S2 = np.delete(arrays_S2, index_S2, axis=0)

    numbers_A = np.delete(numbers_A, number_index)
    labels_A = np.delete(labels_A, number_index)
    arrays_A_fill = np.delete(arrays_A_fill, number_index)

    datapoints.append((number_A, label_A, array_A, number_A, label_S2, array_S2))



In [44]:
i = 0
print(datapoints[i])

(46057, 'Picea_abies', 8497, 46057, 'Picea_abies', array([[[  91,  198,   86, 1375,  401, 1447, 1659, 1695,  700,  316,
           97, 2291],
        [ 121,  225,  137, 1741,  437, 1609, 2363, 2438,  958,  475,
          114, 2287],
        [  78,  168,   72, 1391,  433, 1596, 2328, 2426,  936,  457,
           90, 2271],
        [ 106,  224,  120, 1433,  504, 1622, 1988, 2614, 1049,  425,
          118, 2121],
        [  95,  330,  145, 2229,  490, 1609, 1994, 2607, 1029,  413,
          108, 2091],
        [  94,  222,  132, 1876,  429, 1689, 1868, 1992,  876,  386,
          123, 2110]],

       [[  90,  190,  101, 1488,  297, 1197, 1458, 1619,  557,  215,
          105, 1493],
        [  75,  175,  104, 1651,  309, 1311, 1498, 1787,  655,  298,
          122, 1517],
        [  80,  183,   82, 1651,  277, 1288, 1477, 1766,  637,  286,
           98, 1506],
        [  94,  201,   83, 1558,  307, 1325, 1493, 1547,  640,  291,
          125, 1912],
        [  63,  217,   85, 1598,  296

In [42]:
array_S2 = arrays_S2[index_S2]
print(array_S2)

[[[ 135  339  222 2948  654 2220 2818 3116 1532  643  173 3051]
  [ 113  296  198 2342  658 1937 2583 2785 1496  718  181 3061]
  [ 137  371  248 2650  639 1935 2568 2770 1480  694  163 3038]
  [ 201  423  276 2965  673 1994 2531 3009 1634  728  172 3048]
  [ 229  390  292 2447  655 1985 2534 2992 1623  731  157 3050]
  [ 197  379  290 2581  711 2435 2970 3178 1607  726  182 3220]]

 [[ 148  362  218 2825  651 2182 2746 3102 1491  680  185 3057]
  [ 137  282  227 2193  716 1982 2565 2989 1668  819  195 3074]
  [ 175  403  296 2796  709 1981 2554 2983 1665  809  178 3055]
  [ 227  479  337 3259  846 2393 2800 3197 1914  861  186 3067]
  [ 209  409  340 2812  830 2382 2799 3188 1909  849  171 3053]
  [ 200  389  279 3161  772 2665 3344 3582 1780  829  194 3234]]

 [[ 145  351  200 3135  624 2176 2743 3099 1468  656  164 3048]
  [ 138  309  231 2486  705 1985 2557 2979 1653  806  175 3058]
  [ 185  411  328 2760  706 1965 2539 2962 1648  798  157 3044]
  [ 209  469  361 3241  828 2378 280